# Generate images

In [18]:
#imports
import os
import glob
#restrict gpu usage
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from PIL import Image
import cv2
import bezier
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.patches import Rectangle

# setting theme
#px = 1/plt.rcParams['figure.dpi']
#sns.set(style=None, rc={'figure.figsize':(256*px, 256*px)})

# generate random floating point values
from numpy.random import seed, rand, randint
from random import gauss

# clear folders and files

In [19]:
for i in (['workspace/mobile/images/multi_lines/imgs/*', 
          'workspace/mobile/images/multi_lines/train/*', 
          'workspace/mobile/images/multi_lines/test/*']):
    files = glob.glob(i)
    for f in files:
        os.remove(f)
    
os.remove('workspace/mobile/images/multi_lines/train.csv')
os.remove('workspace/mobile/images/multi_lines/test.csv')

# Generate XML

In [20]:
# for xml generation

import xml.etree.cElementTree as ET

def xmlgen2(foln, fn, path, a_path, width, height, random_nodes, length):

    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder").text = foln
    filename = ET.SubElement(root, "filename").text = fn+'.png'
    path = ET.SubElement(root, "path").text = path
    source = ET.SubElement(root, "source") 
    
    ET.SubElement(source, "database").text = "unknown"
    
    size = ET.SubElement(root, "size")

    ET.SubElement(size, "width").text = width
    ET.SubElement(size, "height").text = height
    
    segmented = ET.SubElement(root, "segmented").text = "0"
    _object = ET.SubElement(root, "_object")
    
    ET.SubElement(_object, "name").text = fn
    ET.SubElement(_object, "pose").text = "Unspecified"
    
    lengths = ET.SubElement(_object, "lengths")
    
    for i,j in enumerate(length):
        ET.SubElement(lengths, "l_{}".format(i)).text = str(j)
        
    ET.SubElement(_object, "truncated").text = "0"
    ET.SubElement(_object, "difficult").text = "0"
    
    bndboxs = ET.SubElement(_object, "bndboxs")
    
    for i,j in enumerate(random_nodes):
        x_0, x_1, x_2, y_0, y_1, y_2 = j
        
        hook = ET.SubElement(bndboxs, "b_{}".format(i))
        ET.SubElement(hook, "x_0").text = str(x_0)
        ET.SubElement(hook, "x_1").text = str(x_1)
        ET.SubElement(hook, "x_2").text = str(x_2)
        ET.SubElement(hook, "y_0").text = str(y_0)
        ET.SubElement(hook, "y_1").text = str(y_1)
        ET.SubElement(hook, "y_2").text = str(y_2)
    
    tree = ET.ElementTree(root)
    tree.write(a_path)

# get all raw images 
def xmlmaker2(arr):
        fn, random_nodes, length = arr
        '''
        x, y = random_nodes
        xmin = str(x[0])
        ymin = str(x[1])
        xmax = str(y[0])
        ymax = str(y[1])
        '''
        width = str(image_width)
        height = str(image_height)

        foln = "./workspace/mobile/images/multi_lines/"
        fn = fn
        path = "{}/imgs/{}.png".format(foln, fn)
        a_path = "{}/imgs/{}.xml".format(foln, fn)
        xmlgen2(foln, fn, path, a_path, width, height, random_nodes, length)
        
#xmlmaker()

# line maker

In [23]:
image_height = image_width = 400
# for 600x600, inches = 8.334,8.334
# for 144x144, inches = 2,2

def multi_line_datasetGen(_, randLines):
        fig = plt.gcf()
        fig.set_size_inches(5.56,5.56)
        ax = plt.subplot()

        ax.set_axis_off()

        plt.axis([0, image_height, 0, image_width])
        plt.tight_layout(pad=0)

        #randLines = np.random.randint(iStart, iStop)
        
        random_nodes = []
        curveLength = []
    
        
        for i in range(randLines):
            r_nodes = np.random.uniform(0,image_width,[2,3])
            
            print(r_nodes)
            con = np.concatenate((r_nodes[0], r_nodes[1]))
            random_nodes.append(con)

            curve = bezier.Curve(r_nodes, degree=2)
            curveLength.append(curve.length)

            curve.plot(num_pts=256, ax=ax)
        #plt.show()
        #print(fig)
            
        
        # Save just the portion _inside_ the second axis's boundaries
        fn = 'li_{}'.format(_)
        ax.figure.savefig('./workspace/mobile/images/multi_lines/imgs/{}.png'.format(fn))

        cap_data = np.array([fn, random_nodes, curveLength], dtype="object")
        xmlmaker2(cap_data)

        #raw.append(cap_data)

        plt.close('all')
        
        
#multi_line_datasetGen("s", 3)

In [24]:
def exec_file_maker():
    counter = -1
    for _ in range(5000):
        counter = counter + 1 if counter < 3 else 0
        #print(counter)
        multi_line_datasetGen(_, counter)

exec_file_maker()

[[304.93784408  63.14235558  72.47246608]
 [331.92668318 283.72988575 224.79213585]]
[[ 60.12382039 232.87030941 397.60541631]
 [209.09477889 379.11169846 195.21398461]]
[[202.68344912 348.86449816 324.47229824]
 [339.84465496 346.06397563 113.30251361]]
[[140.0926582   46.42726409  50.49843805]
 [315.08990617 358.65278997  73.66288628]]
[[240.89319675 224.24870755 311.01168301]
 [236.25712052 102.12480591 277.09206853]]
[[ 83.98766076   4.17521262  96.00185055]
 [387.26034455 185.62772319 316.90261068]]
[[ 23.28133157 197.02534719  83.98764191]
 [227.32024978 301.57695436 249.9279392 ]]
[[194.72920112 122.28822689 279.44645679]
 [363.84398746  61.7312077  301.45899355]]
[[184.0055532  295.28107068 141.72880043]
 [124.4834175  379.42114256 177.62613895]]
[[273.11755022 296.42573891  68.64900445]
 [259.69671542  26.8373885  163.03469843]]
[[ 42.0123841  390.33507498 147.93715183]
 [108.38453292 129.33091952 264.93416985]]
[[153.15182032 363.61346719  15.46870291]
 [231.15951759  85.7701

[[  0.35817524   5.64935207  62.65493888]
 [157.40382551 163.36266313 339.99288596]]
[[ 43.89868464  58.46575173 290.13732526]
 [141.80619095 220.00037963 258.40191681]]
[[393.07812582  43.05900225 186.76087077]
 [227.82585166 275.42612969 393.90438722]]
[[122.38510562 277.24935098 114.76332018]
 [203.95018957 331.43704907 284.73801511]]
[[229.93101092  19.5326619  389.91280578]
 [396.50701744  23.05696817 164.10809089]]
[[380.78814231 128.75190672 151.45410248]
 [396.98334461  91.70008124 377.74045277]]
[[102.12749877 124.96366897 293.71118155]
 [131.35682964 291.04017762  38.60049927]]
[[218.77656149 158.59943068 194.87131405]
 [324.06346584 154.88701524 377.25526696]]
[[ 85.31188454  91.71129954 350.7472312 ]
 [365.5557015  298.06096473 233.57063597]]
[[151.0279956  119.14877882 364.3548947 ]
 [356.56638625 132.71499726 367.23100331]]
[[361.96406113 399.30461026  43.89761562]
 [327.70554585 207.55349909 166.49426748]]
[[186.98119891 236.66723045 205.96500124]
 [361.5439173   68.5222

[[358.77031378 146.6069088  212.40102869]
 [ 23.67751356 182.11731215  43.56411464]]
[[ 93.76427442  63.54050761  57.60589199]
 [398.06385624  36.89654945  20.60134198]]
[[ 14.27586918 270.77518947  36.27664581]
 [ 17.02652419  32.02694863 108.46532305]]
[[399.67946947 268.62942933 371.16492931]
 [156.16864019   2.84580736 347.36125689]]
[[340.10834168 389.06528634 309.12644658]
 [330.07239857  76.38747003 305.82363923]]
[[ 25.2568402  251.42263502 277.30117732]
 [144.5633405  104.43289909 369.57314452]]
[[ 69.53232513 132.98500078 278.1911593 ]
 [364.44816023 224.09585099 129.10987945]]
[[193.9545402  341.41687053 365.90194087]
 [173.94123489 234.45308555 192.38446296]]
[[260.36928584   1.66516197  83.62856579]
 [312.01337456  42.49011914 264.83287226]]
[[180.23856601 273.05772645  50.88491501]
 [ 61.55128397 367.74382595 387.16977223]]
[[ 88.71252728 311.66459751 231.63500065]
 [279.32908445 141.43458478 120.54989267]]
[[331.77972352 200.81131473  81.8446747 ]
 [ 45.76798139  18.3091

[[198.32687059  31.6291894  282.47580723]
 [341.39867452  52.19403375  71.86852508]]
[[322.56001126  98.02382353 292.73803546]
 [ 29.62774455  92.37418332 228.76595897]]
[[ 11.02235427 159.809174    93.12610261]
 [331.44812215 294.36016413 112.11325389]]
[[101.08924312 130.27849333 116.29187528]
 [282.04080642 126.17639377 313.71560723]]
[[ 75.07995451  51.36574798 316.54524838]
 [354.65395598 220.5613787  328.61912922]]
[[260.50877428 390.75085476 188.22120268]
 [201.08703624  23.53047379   1.60247148]]
[[376.86332681 285.4637829  119.20876277]
 [ 63.96507619 398.34648651 156.82095627]]
[[240.36871126 263.16591953 374.34624177]
 [ 45.88886159 167.98625494 190.23220279]]
[[385.35404387 192.23140075 165.75031361]
 [287.65978402  72.14543186 146.51705037]]
[[103.43308643 274.45918672  39.85684145]
 [305.05233666 306.73007828  94.52964002]]
[[157.24649901 143.08902261 192.09946906]
 [359.71293111 350.29192666 199.99824017]]
[[230.95597607 161.22218434  88.34289071]
 [142.14785112 359.1878

[[ 87.86234758 167.3984096  166.02901115]
 [382.38035419 120.84535346 201.95605929]]
[[145.44200052 138.68187489 338.22658481]
 [ 28.99435615 251.22831036 247.41016543]]
[[309.8356952   90.56251901 311.87585212]
 [229.84609606 317.69127526 244.03537484]]
[[235.66748525  13.64776314 301.11013317]
 [253.77035652  34.77784615 261.71218975]]
[[190.84270658  53.83268053 229.07816148]
 [227.04230967 135.41217202 322.02057768]]
[[310.35795969 288.18396969 220.45854158]
 [118.54727212 127.21659223 367.92985216]]
[[354.40255221 318.49078641  37.33021495]
 [ 60.17648149 135.52308279 271.99346825]]
[[386.00360988 389.21703735 378.1266021 ]
 [193.81425493  54.33410307 147.46477981]]
[[321.31340908  28.76412255 270.30331163]
 [385.40058778 113.01784986 308.14560693]]
[[245.28347982 201.31247735 360.74868794]
 [132.10784525 254.0819699  305.77101054]]
[[172.64232341  87.85075821  33.86171931]
 [390.24307815 359.4504099   50.60611665]]
[[179.08461226 247.04657244 258.21866911]
 [272.18459312 281.5103

...

[[282.53752706  79.56133633  69.18158711]
 [ 48.91906503  15.9552281    9.41564516]]
[[184.17253737  90.88445853  90.69116475]
 [ 22.57245563 111.66005924 210.44070068]]
[[123.191516   115.79912938 160.69471583]
 [247.75208707  31.24013831 139.44926755]]
[[174.52629655 308.07410284 108.66453313]
 [ 17.87962227 218.15257695  50.33747298]]
[[237.09946525  51.70336398 319.45528751]
 [202.66322954  22.78261051  65.42409357]]
[[174.32528889 381.36210138  88.40011635]
 [109.36801995 341.51716643  27.10053851]]
[[149.88235683 162.29164596 370.13692573]
 [ 91.01987272 362.60371021 149.98368979]]
[[323.55569588  16.99315049 102.66295406]
 [285.76303551 361.81130309 291.15908436]]
[[163.23861083 207.32628965 275.19965447]
 [110.09991368 394.30423516 159.49212396]]
[[281.35365706 284.37670792 178.03600626]
 [ 71.8095118   30.83797806   7.50162161]]
[[ 32.4730128  174.28840776  36.28590796]
 [338.19735091  67.8398099   67.3522803 ]]
[[276.68557209 309.03918471 177.88081702]
 [ 81.76099183  90.2587

[[ 79.64741205 254.90041024 213.07369596]
 [380.1438993   69.37439872 366.36557049]]
[[ 83.20841897 362.73255997 381.19797665]
 [310.17222839  24.75094595 186.4900732 ]]
[[221.884351   262.32477496  60.20692414]
 [ 10.1920176  185.35400101 209.0341389 ]]
[[ 87.35720707 364.56327137   5.6410146 ]
 [344.67966735 179.31552424  77.68872331]]
[[ 72.69150822 272.14284893 309.19871354]
 [149.27532806  85.67265481 237.79815651]]
[[ 76.56007819 193.94010168 115.91845395]
 [ 48.68649347 239.65205259 288.46190425]]
[[294.24881286 351.19338922 328.34588099]
 [292.22448139 121.129582   136.65888   ]]
[[ 18.05707356 385.20292441 264.47020108]
 [102.61799976 375.76287181 346.38018569]]
[[ 17.19034587 292.73423591  38.69101883]
 [ 16.93673763 192.74511387 171.84647977]]
[[ 24.59473427 337.08285345 359.31625226]
 [ 66.78909804  13.68862343 112.89203167]]
[[184.79316578 120.06029913  95.62638852]
 [ 78.1386765  243.69616237  79.45577149]]
[[160.81921732 397.45310469  75.97331773]
 [344.41946666 372.5253

[[351.73174447 134.07609598   6.09275909]
 [104.63166926 302.66710487  94.50471331]]
[[320.42414474 130.7000164   88.0577415 ]
 [298.40471167 328.757807   149.98995865]]
[[364.3338764  193.86066404 295.76132695]
 [299.23961112 370.08910049 361.59943369]]
[[338.29955092  43.34251399  56.92929599]
 [252.68520799 363.45273458 380.46633797]]
[[133.21007764   1.12202768  53.02756417]
 [130.01550653 362.07727595 302.92217165]]
[[ 51.53615444 317.42577845  87.68887723]
 [340.50999886 314.25514226 330.65244613]]
[[339.17959421 366.96279235 130.85819285]
 [338.9006164  169.22002655 388.39049453]]
[[ 38.13116337  33.29597741  11.04376901]
 [166.76690549   0.84467828 196.12340487]]
[[310.46219126  69.75277327 114.85857038]
 [360.96886516  48.51473107 255.35980071]]
[[146.74833462  53.07633108  17.74498562]
 [331.59024709  32.54428811 315.44964682]]
[[334.75366893 103.91562069  37.52443746]
 [ 93.63363671 143.51930812 284.63304886]]
[[294.33164728 207.98504905 356.40789281]
 [392.20261634 372.4876

[[136.09309498 207.63583608 151.70763826]
 [173.58848354 339.82900127 346.41866333]]
[[376.39622991 206.85279966  29.35551004]
 [275.90773005 386.58658084 218.57269875]]
[[164.21374053 289.65977295 170.9781221 ]
 [ 11.78077181 365.02294666  70.90478278]]
[[175.60234792 256.24841018 166.54395231]
 [  6.10659213 135.9278991  328.97529052]]
[[252.93686252 303.84303858  77.143204  ]
 [ 16.87895293 163.81404163 270.25009227]]
[[189.047897   383.04435288 202.71680253]
 [158.82129394  88.24467864 200.01384616]]
[[307.32283969 344.49223541 225.15332834]
 [303.32925645 155.14824961 250.70322285]]
[[308.2717454  202.31576539  69.60294725]
 [ 42.45693687 305.44355371 378.86660622]]
[[ 13.54121116 191.42452832 356.52902799]
 [216.11896129 115.12300639 271.96236434]]
[[ 71.81066955 386.59854355 338.6970027 ]
 [  0.91652371 179.97579835 290.29428208]]
[[  7.40427779 370.83657364 110.45401379]
 [305.7552436  352.49200801   7.35441259]]
[[129.72774368  63.21422585 289.48804201]
 [277.38649387 152.7761

[[245.35607276 107.32062111 338.07335995]
 [168.90749087 138.76818833 120.38011871]]
[[154.65828028  54.11756963  65.71139037]
 [391.03649808 381.25078549  26.36415684]]
[[313.32661861 230.71184814  33.46359103]
 [115.00213653 362.59443665 126.81905875]]
[[344.75936575 301.8680271  398.10748129]
 [162.47756898   5.40380863 351.28414465]]
[[ 92.07946357 229.80997101 276.01070075]
 [190.46008347 357.26885597  36.58706884]]
[[ 12.30189949 352.88289485  65.99846196]
 [307.83793392 229.20615015 107.69063778]]
[[306.24198124 288.3820551   42.88659223]
 [287.82674104 298.50077835 373.03555389]]
[[231.30823153   1.50680805 131.76267192]
 [281.31824958 152.78413074 311.91941633]]
[[293.73906608 132.82077023 347.38141239]
 [356.97547113 395.48417068 351.60542341]]
[[120.58635042 380.45888983 262.61045233]
 [ 42.14693063 372.6291705  186.88815367]]
[[141.22070467  61.06831149 316.84792033]
 [243.82141356 153.70245102 224.72734206]]
[[384.99024122  77.72280671 295.20031491]
 [ 60.17356127 358.6057

# Partition dataset

In [25]:
# Imports

import os
import re
from shutil import copyfile
import argparse
import math
import random

# Partition images

def partition_images(source, dest, ratio=0.1, copy_xml=True):
    copy_xml = True
    source = source
    dest = dest
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)

    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        filename = images[idx]
        copyfile(os.path.join(source, filename),
                 os.path.join(test_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

    for filename in images:
        copyfile(os.path.join(source, filename),
                 os.path.join(train_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(train_dir, xml_filename))

partition_images("./workspace/mobile/images/multi_lines/imgs", "./workspace/mobile/images/multi_lines")

# XML to CSV

In [26]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET

def xml_to_csv2(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        #print(root.findall('_object'))
        for member in root.findall('_object'):
            # if no line
            if(not member[2]):
                value = (member[0].text,
                     int(float(root.find('size')[0].text)),
                     int(float(root.find('size')[1].text)),
                     str(0),
                     str(0),
                     str(0),
                     str(0),
                     str(0),
                     str(0),
                     str(0),
                     str(0)
                     )
                
                xml_list.append(value)
            else:
                for i,j in enumerate(member[2]):
                    value = (member[0].text,
                         int(float(root.find('size')[0].text)),
                         int(float(root.find('size')[1].text)),
                         member[2][i].text,
                         str(1),
                         float(member[5][i][0].text),
                         float(member[5][i][1].text),
                         float(member[5][i][2].text),
                         float(member[5][i][3].text),
                         float(member[5][i][4].text),
                         float(member[5][i][5].text)
                         )
                    
                    xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'length', 
                   'class', 'x_0', 'x_1', 'x_2', 'y_0', 'y_1', 'y_2',]
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    
    return xml_df

for i in ["test", "train"]:
    examples = xml_to_csv2("workspace/mobile/images/multi_lines/{}".format(i))
    #print(examples)

    examples.to_csv("workspace/mobile/images/multi_lines/{}.csv".format(i), index=None)

In [27]:
def shuffler():
    for i in ["test", "train"]:
        df = pd.read_csv("workspace/mobile/images/multi_lines/{}.csv".format(i))
        ds = df.sample(frac=1)
        ds.to_csv("workspace/mobile/images/multi_lines/{}.csv".format(i), index=None)
        
#shuffler()